## Cassandra Data Modeling 
This project created a pipeline to read event data in CSV format from a local folder, process them in Pandas data frame and write them into Cassandra based on different queries.

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [8]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import csv
import numpy as np
import json
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [9]:
filepath = os.getcwd() + '/event_data'
file_path_list = glob.glob(os.path.join(filepath,'*'))

In [4]:
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
      
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

### Use Pandas to concat csv files

In [5]:
df_list = []
for i in range(len(file_path_list)):
    df_list.append(pd.read_csv(file_path_list[i]))
df = pd.concat(df_list)

In [6]:
df = df.dropna(subset=['artist'])
df_new = df.drop(['auth','method','page','registration','status','ts'], axis=1)

In [7]:
df_new.to_csv('event_datafile_new.csv')

# Part II. The Apache Cassandra coding portion. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# Establish a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the newly created keyspace
session.set_keyspace('udacity')


### Now we need to create tables to run the following queries. 

## Create queries to ask the following three questions of the data

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
## Create table for Query 1
query0 = "CREATE TABLE IF NOT EXISTS  session_lib"
query0 = query0 + " (sessionId int, itemInSession int, artist text, firstName text, gender text, lastName text, length float, level text, location text, song text, userId int, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query0);                    

In [9]:
# Open csv file, write content to table line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO session_lib (sessionId, itemInSession, artist, firstName, gender, lastName, length,level,location,song,userId)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## Assign csv file column elements to columns in Cassendra table, convert datatypes
        session.execute(query1, (int(line[8]),int(line[3]), line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9],int(line[10])))

In [10]:
## Execute SELECT statement to verify the data was entered into the table
query2 = "SELECT artist, song, length FROM session_lib WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(query2)
t = PrettyTable(['artist', 'song', 'length'])
for row in rows:
    t.add_row([row.artist, row.song, row.length])
print(t)

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Create table
query0 = "CREATE TABLE IF NOT EXISTS  user_lib"
query0 = query0 + " (userId int, sessionId int, itemInSession int, artist text, firstName text, gender text, lastName text, length float, level text, location text, song text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query0);
                    

In [12]:
# Open csv file, write content to table line by line
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO user_lib (userId, sessionId, itemInSession, artist, firstName, gender, lastName, length,level,location,song)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## Assign csv file column elements to columns in Cassendra table, convert datatypes
        session.execute(query1, (int(line[10]), int(line[8]), int(line[3]), line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9]))

In [13]:
## Run the SELECT statement to verify the data was entered into the table
query2 = "SELECT artist, song, firstName, lastName FROM user_lib WHERE userId=10 and sessionId=182"
rows = session.execute(query2)
t = PrettyTable(['artist', 'song', 'firstname', 'lastname'])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])
print(t)

+-------------------+------------------------------------------------------+-----------+----------+
|       artist      |                         song                         | firstname | lastname |
+-------------------+------------------------------------------------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
# Create table song_lib
query0 = "CREATE TABLE IF NOT EXISTS  song_lib"
query0 = query0 + " (song text, userId int, sessionId int, artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, PRIMARY KEY (song, userId, sessionId))"
session.execute(query0);

In [17]:
# Open csv file, write content to table line by line
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query1 = "INSERT INTO song_lib (song, userId,sessionId,artist, firstName, gender, itemInSession, lastName, length,level,location)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## Assign csv file column elements to columns in Cassendra table, convert datatypes
        session.execute(query1, (line[9],int(line[10]),int(line[8]),line[0],line[1],line[2],int(line[3]),line[4],float(line[5]),line[6],line[7]))

In [18]:
## Run the SELECT statement to verify the data was entered into the table
query2 = "SELECT firstName, lastName FROM song_lib WHERE song='All Hands Against His Own'"
rows = session.execute(query2)
t = PrettyTable(['firstname', 'lastname'])
for row in rows:
    t.add_row([row.firstname, row.lastname])
print(t)

+------------+----------+
| firstname  | lastname |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
session.execute(" DROP TABLE IF EXISTS session_lib");
session.execute(" DROP TABLE IF EXISTS user_lib");
session.execute(" DROP TABLE IF EXISTS song_lib");

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()